In [211]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#### Group Assignment
- Team Number: <font color="yellow"> 12 </font>
- Team Member Names: <font color="yellow"> Robin Zhang, Jessica Chen, Eric Wang </font>
- Team Strategy Chosen: <font color="red"> Risky </font>

@TODO
- Fix the import
- 

In [212]:
start_date = "2019-11-25"
end_date = "2022-11-25"
targetNumberofStocks = 12
initialInvestment = 500000

In [213]:
#Call the CSV file and store it in a dataframe
tickers = pd.read_csv('Tickers_Example.csv')
length = len(tickers)

#get first value
colname = tickers.columns[0]
missing_data = pd.DataFrame({'AAPL': [colname]})

tickers = tickers.append(missing_data)

tickers.reset_index(inplace=True, drop=True)

#make list of tickers
#tickers is now AAPL because it does not work if tickers is called tickers
#I am not sure why
ticker_list = list(tickers.AAPL)

ticker_data = {}
new_lst = []
i=0

for t in ticker_list:
    firm = yf.Ticker(ticker_list[i])
    firm_hist = firm.history(start=start_date, end=end_date, interval='1mo')

    ticker_data[t] = firm_hist.Volume

    if ticker_data[t].mean() > 200000:
        #make a new lst that stores the values where mean >200 000
        #return that list
        new_lst.append(ticker_list[i])

    i = i + 1

ticker_list = new_lst 
for t in ticker_list:
    if "." in t:
        ticker_list.remove(t)
        
ticker_list.append('^GSPC')
print(ticker_list)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
['ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'AAPL', '^GSPC']


In [214]:
class Stock:

    # Initializing values when class is created
    def __init__(self, ticker, start, end):


        self.ticker = ticker
        self.start = start
        self.end = end
        self.dateRange = pd.date_range(start=self.start, end=self.end, freq=pd.offsets.Week(1))
        self.makeDataDict()
        self.makeDataFrame()
        self.makeColumns()

    # Takes the history of the ticker and puts in dictionary
    def makeDataDict(self):
        self.companies = {}
        self.companies[self.ticker] = yf.Ticker(self.ticker).history(start = self.start, end = self.end)['Close']

    # Creates a DataFrame for every day 1 of the month
    def makeDataFrame(self):
        self.companiesDF = pd.DataFrame(self.companies)
        self.companiesDF = self.companiesDF.groupby(pd.DatetimeIndex(self.companiesDF.index).to_period('W')).nth(0)
        self.companiesDF['pct'] = self.companiesDF[self.ticker].pct_change()
        self.companiesDF.dropna(inplace=True)

    def makeColumns(self):
        self.columns = []
        for column in self.companiesDF.iteritems():
            self.columns.append(column[0])

    # Graphs the price of the Stocks
    def graphStocks(self):
        for x in self.columns:
            plt.plot(self.dateRange, self.companiesDF[x], label = x)
        plt.title("Portfolio Stock Prices", fontsize=20)
        self.makeGraph(True)

    # Adds the Axis and Customization to Graphs
    def makeGraph(self, legend=False):
        plt.xticks(rotation=70)
        plt.xlabel("Dates", fontsize=14)
        plt.ylabel("Price $(USD)", fontsize=14)
        if legend:
            plt.legend(bbox_to_anchor=(1.0, 1.0))

In [215]:
def makeObjects(tickerList):
    objectList = []
    for ticker in tickerList:
        objectList.append(Stock(ticker, start_date, end_date))
    return objectList
    
stockObjects = makeObjects(ticker_list)

In [216]:
def findHighestStd(tickerList):
    '''
    purpose:
        calculates beta between each individual stock and the overall market (SP500 index)
    input:
        tickerList (listof Stock): a list of Stock objects
    output: 
        A list of the beta of each stock in tickerList (assume US market)
    '''

    # find stock object with highest std
    highestStd = 0
    highestStdObj = tickerList[0]

    for ticker in tickerList:
        if(ticker.ticker != '^GSPC'):
            tempStd = ticker.companiesDF['pct'].std()
            if(tempStd >= highestStd):
                highestStd = tempStd
                highestStdObj = ticker

    return highestStdObj

mainStock = findHighestStd(stockObjects)
mainStock.ticker

'OXY'

In [217]:
# currently not working with class objects
def findHighestBeta(tickerList):
    '''
    purpose:
        calculates beta between each individual stock and the overall market (SP500 index)
    input:
        tickerList (listof Stock): a list of Stock objects
    output: 
        A list of the beta of each stock in tickerList (assume US market)
    '''

    # calculating market variance
    marketVar=tickerList[-1].companiesDF['pct'].var()
    
    # calculating each Beta, then store in the beta dict
    highestBeta = 0
    highestBetaObj = tickerList[0]
    for ticker in tickerList:
        
        if(ticker.ticker != '^GSPC'):
            temp=pd.DataFrame()
            temp['market_close']=tickerList[-1].companiesDF['pct']
            temp["ticker_close"]=ticker.companiesDF['pct']

            tempBeta=temp.cov()/marketVar
            tempBeta=tempBeta.iloc[0,1]
            tempBeta=np.absolute(tempBeta)

            if(tempBeta>highestBeta):
                highestBeta = tempBeta
                highestBetaObj = ticker
                print(highestBeta)

    return highestBetaObj

highBetaStock = findHighestBeta(stockObjects)
highBetaStock.ticker


0.6905668717359826
0.8474692642385345
1.0167705074523892
1.5934050847510521
2.0603057934352695


'BA'

In [218]:
def calculateCorrelation(ticker1, ticker2):
    '''
    purpose:
        calculates the correlation between two stock
    input:
        ticker1 (Stock): Stock object 1
        ticker2 (Stock): Stock object 2
    output: 
        A Dictionary with key of format "ticker1+ticker2" and the 
        value is the correlation between the 2 tickers
    '''
    correlation=pd.DataFrame()
    correlation[ticker1.ticker]=ticker1.companiesDF['pct']
    correlation[ticker2.ticker]=ticker2.companiesDF['pct']
    correlation=correlation.corr()
    correlation=correlation.iloc[0,1]


    return correlation

In [228]:
def selectPortfolio(mainStock, tickerList):
    portfolio = [mainStock]
    
    threshold = 0.95  
    while True:
        for ticker in tickerList:
            if ticker.ticker != mainStock.ticker:
                if calculateCorrelation(mainStock, ticker) >= threshold:
                    portfolio.append(ticker) 
        if len(portfolio) < targetNumberofStocks:
                portfolio = [mainStock]
                threshold -= 0.05
        elif len(portfolio) > targetNumberofStocks:
                portfolio = [mainStock]
                threshold += 0.001
        else:
            break
    return portfolio

portfolioObj = selectPortfolio(mainStock, stockObjects)
for ticker in portfolioObj:
    if(ticker.ticker != mainStock.ticker):
        print(ticker.ticker, calculateCorrelation(mainStock, ticker))

AIG 0.5506601425914787
BA 0.5530096323128921
BAC 0.5778106992060774
C 0.5975415346692405
CAT 0.5410165241775471
COP 0.7598450571864305
JPM 0.5613003693998049
KMI 0.6843432740581594
SLB 0.7926146430157
SPG 0.65449708377271
USB 0.6080975156800223


In [223]:
def portfolioFinal(portfolioObj):
    tempDict = dict()
    tempPrice = []
    tempShares = []
    tempValue = []
    tempWeight = []
    tempTicker = []

    for ticker in portfolioObj:
        # @TODO change it to 11-25 to 11-26
        price = list(yf.Ticker(ticker.ticker).history(start = "2022-11-23", end = "2022-11-24", interval='1d')['Close'])[0]
        tempPrice.append(price)
        weight = 0

        if(ticker.ticker != mainStock.ticker):
            weight = (100/(2*targetNumberofStocks))/100
        else:
            weight = (100 - (targetNumberofStocks-1)*100/(2*targetNumberofStocks))/100

        tempWeight.append(weight)
        value = initialInvestment*weight
        tempValue.append(value)
        tempShares.append(value/price)
        tempTicker.append(ticker.ticker)

    tempDict = {"Ticker":tempTicker, "Price":tempPrice, "Shares":tempShares, 
    "Value":tempValue, "Weight":tempWeight, }
    Portfolio_Final = pd.DataFrame(tempDict)

    Portfolio_Final.index += 1

    return Portfolio_Final

Portfolio_Final = portfolioFinal(portfolioObj)
Portfolio_Final

,Ticker,Price,Shares,Value,Weight
1,OXY,71.120003,3808.117588,270833.333333,0.541667
2,AIG,60.959999,341.754161,20833.333333,0.041667
3,BA,174.850006,119.149743,20833.333333,0.041667
4,BAC,37.610001,553.930683,20833.333333,0.041667
5,C,48.240002,431.868421,20833.333333,0.041667
6,CAT,236.919998,87.934043,20833.333333,0.041667
7,COP,126.739998,164.378520,20833.333333,0.041667
8,JPM,136.479996,152.647523,20833.333333,0.041667
9,KMI,18.650000,1117.068834,20833.333333,0.041667
10,SLB,51.419998,405.160134,20833.333333,0.041667


In [221]:
def outputCSV(Portfolio_Final):
    Stocks_Group_12 = pd.DataFrame(Portfolio_Final[['Ticker', 'Shares']])
    Stocks_Group_12.to_csv("Stocks_Group_12.csv")
    return Stocks_Group_12

Stocks_Group_12 = outputCSV(Portfolio_Final)
Stocks_Group_12


,Ticker,Shares
1,OXY,3808.117588
2,AIG,341.754161
3,BA,119.149743
4,BAC,553.930683
5,C,431.868421
6,CAT,87.934043
7,COP,164.378520
8,JPM,152.647523
9,KMI,1117.068834
10,SLB,405.160134


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

- Robin:
- Jessica:
- Eric: 